In [123]:
import mysql.connector
from datetime import datetime
import mysql.connector
import pandas as pd
from math import *
import numpy as np

In [2]:
# Faire moyenne des votes influenceurs dans la journée
# Test de moyenne théorique : moy - moy tout le monde / ecart type/racine(n)
# si supérieur à p-value -> influenceur est chaud
# par catégorie

Get today's date

In [11]:
date = datetime.today().strftime('%Y-%m-%d')
print(date)

2021-03-24


Get number of today's votes for all influenceurs

In [13]:
print("SELECT influenceur.id_influenceur, influenceur.nom_influenceur, COUNT(voter.id_vote) FROM influenceur, voter WHERE voter.id_influenceur=influenceur.id_influenceur AND voter.date_vote=\""+date+"\" GROUP BY voter.id_influenceur")

SELECT influenceur.id_influenceur, influenceur.nom_influenceur, COUNT(voter.id_vote) FROM influenceur, voter WHERE voter.id_influenceur=influenceur.id_influenceur AND voter.date_vote="2021-03-24" GROUP BY voter.id_influenceur


In [86]:
nb_today=[]

config = {
        'user': 'root',
        'password': '',
        'host': 'localhost',
        'database': 'abdia_bon'
    }

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

select_vote = ("SELECT influenceur.id_influenceur, influenceur.nom_influenceur, influenceur.id_categorie, COUNT(voter.id_vote) FROM influenceur, voter WHERE voter.id_influenceur=influenceur.id_influenceur AND voter.date_vote=\""+date+"\" GROUP BY voter.id_influenceur")
cursor.execute(select_vote)
    
result = cursor.fetchall()

for ligne in result:
    nb_today.append(ligne)

In [87]:
todays = pd.DataFrame(nb_today, columns=['id_influenceur', 'nom_influenceur', 'id_categorie', 'nb_votes'])

In [88]:
todays.head()

,id_influenceur,nom_influenceur,id_categorie,nb_votes
0,1,DUDE THAT COOKZ,5,13
1,2,PAPA SCRIPT,10,20
2,3,MICHAEL BLACKSON,6,24
3,4,INVESTIR AU PAYS,7,20
4,5,WENDY SHAY,9,25


In [62]:
print("SELECT categorie.id_categorie, categorie.nom_categorie, COUNT(voter.id_vote) FROM influenceur, voter, categorie WHERE voter.id_influenceur=influenceur.id_influenceur AND categorie.id_categorie=influenceur.id_categorie AND voter.date_vote=\""+date+"\" GROUP BY influenceur.id_categorie")

SELECT categorie.id_categorie, categorie.nom_categorie, COUNT(voter.id_vote) FROM influenceur, voter, categorie WHERE voter.id_influenceur=influenceur.id_influenceur AND categorie.id_categorie=influenceur.id_categorie AND voter.date_vote="2021-03-24" GROUP BY influenceur.id_categorie


In [76]:
nb_today_cat=[]

config = {
        'user': 'root',
        'password': '',
        'host': 'localhost',
        'database': 'abdia_bon'
    }

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

select_vote_cat = ("SELECT categorie.id_categorie, categorie.nom_categorie, COUNT(voter.id_vote) FROM influenceur, voter, categorie WHERE voter.id_influenceur=influenceur.id_influenceur AND categorie.id_categorie=influenceur.id_categorie AND voter.date_vote=\""+date+"\" GROUP BY influenceur.id_categorie")
cursor.execute(select_vote_cat)
    
result = cursor.fetchall()

for ligne in result:
    nb_today_cat.append(ligne)

In [80]:
todays_cat = pd.DataFrame(nb_today_cat, columns=['id_categorie', 'nom_categorie', 'nb_votes'])

In [81]:
todays_cat

,id_categorie,nom_categorie,nb_votes
0,1,Travel,121
1,2,Lifestyle,108
2,3,Fashion Homme,124
3,4,Fashion Femme,133
4,5,Food,123
5,6,Humour,143
6,7,Media d'influence,118
7,8,Chanteurs,126
8,9,Chanteuses,137
9,10,Danseurs,146


In [94]:
todays.iloc[0, 2] == todays_cat.iloc [4,0]

True

In [98]:
list_votes_cat=[]
for i in range(0, len(todays)):
    for j in range (0, len(todays_cat)):
        if todays.iloc[i, 2] == todays_cat.iloc[j,0]:
            list_votes_cat.append(todays_cat.iloc[j,2])

In [100]:
len(list_votes_cat)

78

In [101]:
todays = todays.assign(votes_cat = list_votes_cat)

In [110]:
todays.head()

,id_influenceur,nom_influenceur,id_categorie,nb_votes,votes_cat,moyenne
0,1,DUDE THAT COOKZ,5,13,123,0.105691
1,2,PAPA SCRIPT,10,20,146,0.136986
2,3,MICHAEL BLACKSON,6,24,143,0.167832
3,4,INVESTIR AU PAYS,7,20,118,0.169492
4,5,WENDY SHAY,9,25,137,0.182482


In [145]:
temp = todays[['id_categorie', 'nb_votes']]

In [148]:
temp.head()

,id_categorie,nb_votes
0,5,13
1,10,20
2,6,24
3,7,20
4,9,25


In [151]:
grouped_data = temp.groupby('id_categorie')

In [152]:
for group in grouped_data.groups:
  print(group)

1
2
3
4
5
6
7
8
9
10
11
12
13


In [198]:
moy_cat=temp.groupby('id_categorie').mean()['nb_votes']

In [199]:
moy_cat

id_categorie
1     20.166667
2     18.000000
3     20.666667
4     22.166667
5     20.500000
6     23.833333
7     19.666667
8     21.000000
9     22.833333
10    24.333333
11    20.833333
12    21.333333
13    23.166667
Name: nb_votes, dtype: float64

In [200]:
ecart_type=temp.groupby('id_categorie').std()['nb_votes']

In [201]:
ecart_type

id_categorie
1     6.337718
2     4.381780
3     4.273952
4     4.996666
5     6.655825
6     3.430258
7     2.732520
8     3.033150
9     2.316607
10    6.153590
11    4.445972
12    4.966555
13    3.430258
Name: nb_votes, dtype: float64

In [202]:
list_moy_cat = list(moy_cat)

In [203]:
list_moy_cat

[20.166666666666668,
 18.0,
 20.666666666666668,
 22.166666666666668,
 20.5,
 23.833333333333332,
 19.666666666666668,
 21.0,
 22.833333333333332,
 24.333333333333332,
 20.833333333333332,
 21.333333333333332,
 23.166666666666668]

In [184]:
list_moy_cat[4]

20.5

In [204]:
list_ecart_type = list(ecart_type)

In [205]:
list_moy = []

In [206]:
for i in range(todays.shape[0]):
    x = todays.loc[i, 'id_categorie']-1
    #print(x)
    list_moy.append(list_moy_cat[x])

In [207]:
list_moy

[20.5,
 24.333333333333332,
 23.833333333333332,
 19.666666666666668,
 22.833333333333332,
 21.333333333333332,
 20.166666666666668,
 22.166666666666668,
 22.166666666666668,
 20.5,
 20.166666666666668,
 23.166666666666668,
 18.0,
 21.0,
 20.666666666666668,
 20.5,
 21.0,
 20.666666666666668,
 24.333333333333332,
 21.333333333333332,
 18.0,
 20.166666666666668,
 20.666666666666668,
 18.0,
 18.0,
 21.0,
 20.5,
 20.166666666666668,
 20.666666666666668,
 22.166666666666668,
 22.166666666666668,
 22.166666666666668,
 23.833333333333332,
 20.666666666666668,
 20.166666666666668,
 21.0,
 22.833333333333332,
 18.0,
 20.5,
 20.833333333333332,
 22.833333333333332,
 22.166666666666668,
 24.333333333333332,
 21.333333333333332,
 22.833333333333332,
 20.5,
 24.333333333333332,
 20.833333333333332,
 20.666666666666668,
 18.0,
 21.0,
 21.333333333333332,
 21.333333333333332,
 19.666666666666668,
 22.833333333333332,
 20.166666666666668,
 19.666666666666668,
 21.0,
 22.833333333333332,
 24.333333333

In [208]:
list_ecart_type = list(ecart_type)
list_ec_cat=[]

In [209]:
for i in range(todays.shape[0]):
    x = todays.loc[i, 'id_categorie']-1
    #print(x)
    list_ec_cat.append(list_ecart_type[x])

In [210]:
list_ec_cat

[6.6558245169174945,
 6.153589738247641,
 3.4302575219167823,
 2.7325202042558927,
 2.316606713852541,
 4.966554808583779,
 6.3377177806105145,
 4.996665554814197,
 4.996665554814197,
 6.6558245169174945,
 6.3377177806105145,
 3.4302575219167823,
 4.381780460041329,
 3.03315017762062,
 4.273952113286562,
 6.6558245169174945,
 3.03315017762062,
 4.273952113286562,
 6.153589738247641,
 4.966554808583779,
 4.381780460041329,
 6.3377177806105145,
 4.273952113286562,
 4.381780460041329,
 4.381780460041329,
 3.03315017762062,
 6.6558245169174945,
 6.3377177806105145,
 4.273952113286562,
 4.996665554814197,
 4.996665554814197,
 4.996665554814197,
 3.4302575219167823,
 4.273952113286562,
 6.3377177806105145,
 3.03315017762062,
 2.316606713852541,
 4.381780460041329,
 6.6558245169174945,
 4.445971959725643,
 2.316606713852541,
 4.996665554814197,
 6.153589738247641,
 4.966554808583779,
 2.316606713852541,
 6.6558245169174945,
 6.153589738247641,
 4.445971959725643,
 4.273952113286562,
 4.381780

In [211]:
len(list_moy)

78

In [212]:
todays['moy_cat']=list_moy

In [213]:
todays['ec_cat']=list_ec_cat

In [214]:
todays.head()

,id_influenceur,nom_influenceur,id_categorie,nb_votes,votes_cat,moyenne,moy_cat,ec_cat
0,1,DUDE THAT COOKZ,5,13,123,21.423077,20.500000,6.655825
1,2,PAPA SCRIPT,10,20,146,21.423077,24.333333,6.153590
2,3,MICHAEL BLACKSON,6,24,143,21.423077,23.833333,3.430258
3,4,INVESTIR AU PAYS,7,20,118,21.423077,19.666667,2.732520
4,5,WENDY SHAY,9,25,137,21.423077,22.833333,2.316607


In [215]:
todays['test'] = (todays.nb_votes - todays.moy_cat)/(todays.ec_cat / np.sqrt(todays.shape[0]-1))

In [216]:
todays.head()

,id_influenceur,nom_influenceur,id_categorie,nb_votes,votes_cat,moyenne,moy_cat,ec_cat,test
0,1,DUDE THAT COOKZ,5,13,123,21.423077,20.500000,6.655825,-9.887916
1,2,PAPA SCRIPT,10,20,146,21.423077,24.333333,6.153590,-6.179295
2,3,MICHAEL BLACKSON,6,24,143,21.423077,23.833333,3.430258,0.426351
3,4,INVESTIR AU PAYS,7,20,118,21.423077,19.666667,2.732520,1.070436
4,5,WENDY SHAY,9,25,137,21.423077,22.833333,2.316607,8.207014


In [217]:
qnorm = 1.959964

In [219]:
list_signif=[]

In [220]:
for i in range(todays.shape[0]):
    if todays.loc[i, 'test'] > qnorm:
        list_signif.append(1)
    elif todays.loc[i, 'test'] < -1*qnorm:
        list_signif.append(-1)
    else:
        list_signif.append(0)

In [222]:
todays['signif']=list_signif

In [226]:
todays.head()

,id_influenceur,nom_influenceur,id_categorie,nb_votes,votes_cat,moy_cat,ec_cat,test,signif
0,1,DUDE THAT COOKZ,5,13,123,20.500000,6.655825,-9.887916,-1
1,2,PAPA SCRIPT,10,20,146,24.333333,6.153590,-6.179295,-1
2,3,MICHAEL BLACKSON,6,24,143,23.833333,3.430258,0.426351,0
3,4,INVESTIR AU PAYS,7,20,118,19.666667,2.732520,1.070436,0
4,5,WENDY SHAY,9,25,137,22.833333,2.316607,8.207014,1
